In [3]:
import pandas as pd
import numpy as np

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
weather = pd.read_csv('data/weather.csv')
spray = pd.read_csv('data/spray.csv')

In [2]:
%ls

Clean Kaggle JB.ipynb      README.md
Ike-Kaggle JB Edits.ipynb  data/
Project3JCode.ipynb        df_distance_updated.csv


In [4]:
#creating columns with airport coords
train['lat_mid'] = 41.786839 
train['long_mid'] = -87.752090
train['lat_ohare'] = 41.972582
train['long_ohare'] = -87.908753

In [5]:
#using the vincenty method to determine distance in miles between locations and airports
from geopy.distance import vincenty
train['mid']=train.apply(lambda x: vincenty((x['Latitude'], x['Longitude']), (x['lat_mid'], x['long_mid'] )).miles, axis=1)
train['ohare']=train.apply(lambda x: vincenty((x['Latitude'], x['Longitude']), (x['lat_ohare'], x['long_ohare'] )).miles, axis=1)


In [6]:
#Label each entry with its closest airport 'Station'
train.ix[train['mid'] > train['ohare'], 'Station'] = 1 #map 1 to closer if Ohare is closer
train.ix[train['mid'] < train['ohare'], 'Station'] = 2 #map 2 if closer to Midway
train.Station = train.Station.astype(int) #convert from float to int

In [7]:
#fixing missing values in weather dataframe
weather.replace('M',np.NaN, inplace=True) #replace 'M' with 'NaN'
weather.fillna(method='ffill', inplace=True) #fill in 'NaN' values with those from above it



In [8]:
#merge train and weather dataframes
#merge based on matching Date and Station and value
train1 = train.merge(weather, on= ['Date', 'Station'])

In [9]:
train1.columns

Index([u'Date', u'Address', u'Species', u'Block', u'Street', u'Trap',
       u'AddressNumberAndStreet', u'Latitude', u'Longitude',
       u'AddressAccuracy', u'NumMosquitos', u'WnvPresent', u'lat_mid',
       u'long_mid', u'lat_ohare', u'long_ohare', u'mid', u'ohare', u'Station',
       u'Tmax', u'Tmin', u'Tavg', u'Depart', u'DewPoint', u'WetBulb', u'Heat',
       u'Cool', u'Sunrise', u'Sunset', u'CodeSum', u'Depth', u'Water1',
       u'SnowFall', u'PrecipTotal', u'StnPressure', u'SeaLevel',
       u'ResultSpeed', u'ResultDir', u'AvgSpeed'],
      dtype='object')

In [11]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats

In [ ]:
dtc3 = DecisionTreeClassifier(max_depth=3)


In [16]:
train1['Heat']#.value_counts()

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       9
26       9
27       9
28       9
29       9
        ..
10476    0
10477    0
10478    0
10479    0
10480    0
10481    0
10482    0
10483    0
10484    0
10485    0
10486    0
10487    0
10488    0
10489    0
10490    0
10491    0
10492    0
10493    0
10494    0
10495    0
10496    0
10497    0
10498    0
10499    0
10500    0
10501    0
10502    0
10503    0
10504    0
10505    0
Name: Heat, dtype: object

In [17]:
train1.DewPoint

0        58
1        58
2        58
3        58
4        58
5        58
6        58
7        59
8        59
9        59
10       59
11       59
12       59
13       59
14       59
15       59
16       59
17       59
18       59
19       59
20       59
21       59
22       59
23       59
24       59
25       48
26       48
27       48
28       48
29       48
         ..
10476    52
10477    52
10478    52
10479    52
10480    52
10481    52
10482    52
10483    52
10484    52
10485    52
10486    52
10487    52
10488    52
10489    52
10490    52
10491    52
10492    52
10493    52
10494    52
10495    52
10496    52
10497    52
10498    52
10499    52
10500    52
10501    52
10502    52
10503    52
10504    52
10505    52
Name: DewPoint, dtype: int64

In [10]:
train1.head(5)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,BR HZ,0,NaN,0.0,0.00,29.39,30.11,5.8,18,6.5
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,...,BR HZ,0,NaN,0.0,0.00,29.39,30.11,5.8,18,6.5
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,...,BR HZ,0,NaN,0.0,0.00,29.39,30.11,5.8,18,6.5
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,BR HZ,0,NaN,0.0,0.00,29.39,30.11,5.8,18,6.5
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,...,BR HZ,0,NaN,0.0,0.00,29.39,30.11,5.8,18,6.5
